In [1]:
pip install -U scikit-fuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt

from sklearn import datasets
from fcmeans import FCM

In [3]:
df = pd.read_csv('credit_card.csv', header = 1)

In [4]:
print(df.shape)

(30000, 25)


In [5]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [6]:
df['BILL_TOTAL']=df['BILL_AMT1']+df['BILL_AMT2']+df['BILL_AMT3']+df['BILL_AMT4']+df['BILL_AMT5']+df['BILL_AMT6']

In [7]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,BILL_TOTAL
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,689,0,0,0,0,1,7704
1,2,120000,2,2,2,26,-1,2,0,0,...,3455,3261,0,1000,1000,1000,0,2000,1,17077
2,3,90000,2,2,2,34,0,0,0,0,...,14948,15549,1518,1500,1000,1000,1000,5000,0,101653
3,4,50000,2,2,1,37,0,0,0,0,...,28959,29547,2000,2019,1200,1100,1069,1000,0,231334
4,5,50000,1,2,1,57,-1,0,-1,0,...,19146,19131,2000,36681,10000,9000,689,679,0,109339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,31237,15980,8500,20000,5003,3047,5000,1000,0,725349
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,5190,0,1837,3526,8998,129,0,0,0,21182
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20582,19357,0,0,22000,4200,2000,3100,1,70496
29998,29999,80000,1,3,1,41,1,-1,0,0,...,11855,48944,85900,3409,1178,1926,52964,1804,1,266611


In [8]:
new_df=df.iloc[:, [1, 25]].values

In [9]:
new_df

array([[ 20000,   7704],
       [120000,  17077],
       [ 90000, 101653],
       ...,
       [ 30000,  70496],
       [ 80000, 266611],
       [ 50000, 230874]])

In [10]:
max=new_df.max()
new_df=new_df/max

In [11]:
new_df

array([[0.00379948, 0.00146356],
       [0.02279686, 0.00324418],
       [0.01709764, 0.01931141],
       ...,
       [0.00569921, 0.01339239],
       [0.01519791, 0.05064911],
       [0.00949869, 0.04386002]])

In [12]:
# colors = ['b', 'orange', 'g', 'r', 'c', 'm', 'y', 'k', 'Brown', 'ForestGreen']
# fig1, axes1 = plt.subplots(3, 3, figsize=(8, 8))
# fpcs = []

# for ncenters, ax in enumerate(axes1.reshape(-1), 2):
#     cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
#         new_df, ncenters, 2, error=0.005, maxiter=1000, init=None)
#     # Store fpc values for later
#     fpcs.append(fpc)

#     # Plot assigned clusters, for each data point in training set
#     cluster_membership = np.argmax(u, axis=0)
#     for j in range(ncenters):
#         ax.plot(new_df[0, u.argmax(axis=0) == j],
#                 new_df[1, u.argmax(axis=0) == j], '.', color=colors[j])

#     # # Mark the center of each fuzzy cluster
#     for pt in cntr:
#         ax.plot(pt[0], pt[1], 'rs')

#     ax.set_title('Centers = {0}; FPC = {1:.2f}'.format(ncenters, fpc))
#     ax.axis('off')

# fig1.tight_layout()

In [13]:
# fcmean=FCM(n_clusters=4)
# fcmean.fit(new_df)
# cnt=fcmean.centers
# pred=fcmean.predict(new_df)


In [14]:
# plt.scatter(new_df[pred == 0, 0], new_df[pred == 0, 1], s = 10, c = 'r')
# plt.scatter(new_df[pred == 1, 0], new_df[pred == 1, 1], s = 10, c = 'b' )
# plt.scatter(new_df[pred == 2, 0], new_df[pred == 2, 1], s = 10, c = 'g' )
# plt.scatter(cnt[:, 0], cnt[:, 1], s = 300, c = 'black',marker = '+')
# plt.title('Wine Clustering')
# plt.show()